In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Function to load and resize images """
def load_and_resize_images(image_paths, size=(224, 224), is_mask=False):
    images = []
    for path in tqdm(image_paths, desc="Resizing images" if not is_mask else "Resizing masks"):
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE if is_mask else cv2.IMREAD_COLOR)  # Load mask in grayscale, image in color
        img = cv2.resize(img, size)  # Resize image/mask to 512x512
        if not is_mask:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for images
        else:
            img = np.expand_dims(img, axis=-1)  # Add channel dimension for masks
        images.append(img)
    return np.array(images, dtype='float32')

""" Function to load dataset and split into train, validation, and test sets """
def load_data(path):
    """ Loading the images and masks """
    X = sorted(glob(os.path.join(path, "images", "*.jpg")))
    Y = sorted(glob(os.path.join(path, "masks", "*.jpg")))

    """ Splitting the data into 80% training, 10% validation, 10% testing """
    train_x, temp_x, train_y, temp_y = train_test_split(X, Y, test_size=0.2, random_state=42)
    val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=42)

    """ Resize images and masks """
    train_x = load_and_resize_images(train_x)
    val_x = load_and_resize_images(val_x)
    test_x = load_and_resize_images(test_x)

    train_y = load_and_resize_images(train_y, is_mask=True)
    val_y = load_and_resize_images(val_y, is_mask=True)
    test_y = load_and_resize_images(test_y, is_mask=True)

    """ Normalize images and masks to the range [0, 1] """
    train_x = train_x / 255.0
    val_x = val_x / 255.0
    test_x = test_x / 255.0

    train_y = train_y / 255.0
    val_y = val_y / 255.0
    test_y = test_y / 255.0

    return (train_x, train_y), (val_x, val_y), (test_x, test_y)

In [ ]:

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)

    """ Load the dataset """
    data_path = "Kvasir-SEG/Kvasir-SEG"
    (train_x, train_y), (val_x, val_y), (test_x, test_y) = load_data(data_path)

    print(f"Train:	 {len(train_x)} - {len(train_y)}")
    print(f"Validation:	 {len(val_x)} - {len(val_y)}")
    print(f"Test:	 {len(test_x)} - {len(test_y)}")

    """ Debugging: Check shapes and data types """
    print("Train X shape:", train_x.shape)
    print("Train Y shape:", train_y.shape)
    print("Validation X shape:", val_x.shape)
    print("Validation Y shape:", val_y.shape)
    print("Test X shape:", test_x.shape)
    print("Test Y shape:", test_y.shape)

    print("Train X dtype:", train_x.dtype)
    print("Train Y dtype:", train_y.dtype)

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

def display_image_mask_pairs(images, masks, title, num_images=10):
    """ Display a set of images with their corresponding masks (from preloaded arrays) """
    plt.figure(figsize=(5, 5))
    
    for i in range(min(num_images, len(images))):
        img = images[i]  # Image array (already loaded and normalized)
        mask = masks[i]  # Mask array (already loaded and normalized)
        
        # Plot original image
        plt.subplot(num_images, 2, 2 * i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{title} - Image {i+1}")
        
        # Plot mask
        plt.subplot(num_images, 2, 2 * i + 2)
        plt.imshow(mask.squeeze(), cmap="gray")  # Remove channel dimension for display
        plt.axis('off')
        plt.title(f"{title} - Mask {i+1}")
    
    plt.tight_layout()
    plt.show()

# Display using preloaded arrays (no file paths)
display_image_mask_pairs(train_x, train_y, "Training Set", 2)
display_image_mask_pairs(val_x, val_y, "Validation Set", 2)
display_image_mask_pairs(test_x, test_y, "Testing Set", 2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation, MaxPool2D,
    Conv2DTranspose, Concatenate, Input, AveragePooling2D,
    GlobalAveragePooling2D, UpSampling2D, Reshape, Dense
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import backend as K

def ASPP(inputs):
    """ Atrous Spatial Pyramid Pooling (ASPP) """
    shape = K.int_shape(inputs)  # Use Keras static shape instead of tf.shape
    h, w = shape[1], shape[2]

    y1 = AveragePooling2D(pool_size=(h, w))(inputs)
    y1 = Conv2D(256, 1, padding="same", use_bias=False)(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((h, w), interpolation="bilinear")(y1)

    y2 = Conv2D(256, 1, padding="same", use_bias=False)(inputs)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=6)(inputs)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=12)(inputs)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=18)(inputs)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])
    y = Conv2D(256, 1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def deeplabv3_plus(shape):
    """ Input """
    inputs = Input(shape=shape)

    """ Encoder using MobileNetV2 """
    encoder = MobileNetV2(weights="imagenet", include_top=False, input_tensor=inputs)
    
    image_features = encoder.get_layer("block_13_expand_relu").output  # High-level features
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)

    x_b = encoder.get_layer("block_3_expand_relu").output  # Low-level features
    x_b = Conv2D(filters=48, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)

    x = Concatenate()([x_a, x_b])

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    x = Conv2D(1, 1)(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs, x)
    return model

if __name__ == "__main__":
    model = deeplabv3_plus((224, 224, 3))
    #model.summary()

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
# **Intersection over Union (IoU)**
def iou(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return intersection / (union + K.epsilon())

# **Dice Coefficient**
def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

# **Precision**
def precision(y_true, y_pred):
    true_positive = K.sum(K.round(y_true * y_pred))
    predicted_positive = K.sum(K.round(y_pred))
    return true_positive / (predicted_positive + K.epsilon())

# **Recall**
def recall(y_true, y_pred):
    true_positive = K.sum(K.round(y_true * y_pred))
    possible_positive = K.sum(K.round(y_true))
    return true_positive / (possible_positive + K.epsilon())


# **Binary Accuracy**
def accuracy(y_true, y_pred):
    correct_predictions = K.sum(K.cast(K.equal(K.round(y_true), K.round(y_pred)), dtype="float32"))
    total_predictions = K.cast(K.prod(K.shape(y_true)), dtype="float32")
    return correct_predictions / (total_predictions + K.epsilon())


# **Focal Loss**
def focal_loss(alpha=0.8, gamma=2.0):
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, 1e-7, 1.0 - 1e-7)  # Avoid log(0)
        focal_weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(focal_weight * K.binary_crossentropy(y_true, y_pred))
    return loss

# **Dice Loss**
def dice_loss(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def mixed_loss(y_true, y_pred):
    return focal_loss()(y_true, y_pred) + dice_loss(y_true, y_pred)

initial_learning_rate = 0.0005  
decay_steps = 1000  

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
)

optimizer = tf.keras.optimizers.AdamW(learning_rate=lr_schedule)

model.compile(optimizer=optimizer,
              loss=mixed_loss,
              metrics=[accuracy, precision, recall, dice_coefficient, iou])

batch_size = 4  
epochs = 10


history = model.fit(
    train_x, train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_x, val_y),
    verbose=1
)

In [ ]:
predictions = model.predict(test_x[:5])
plt.figure(figsize=(15, 5))
for i in range(5):
    plt.subplot(3, 5, i + 1)
    plt.imshow(test_x[i])
    plt.title("Image")
    plt.axis('off')
    
    plt.subplot(3, 5, i + 6)
    plt.imshow(test_y[i].squeeze(), cmap='gray')
    plt.title("True Mask")
    plt.axis('off')
    
    plt.subplot(3, 5, i + 11)
    plt.imshow(predictions[i].squeeze() > 0.5, cmap='gray')
    plt.title("Predicted Mask")
    plt.axis('off')
plt.tight_layout()
plt.show()

# save

In [ ]:
model.save('models\deeplabv3_plus&mobilenet_kvasir.h5')